In [2]:
import os
import base64
import re
import csv
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import *
from azure.search.documents.models import VectorizedQuery
import openai
from openai import AzureOpenAI

In [3]:
load_dotenv()

# Get Environment settings from .env file
load_dotenv()

# Azure AI Search Index Settings
service_endpoint = f"{os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT')}"
index_creds = AzureKeyCredential(os.getenv("AZURE_SEARCH_INDEX_KEY"))
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")

## Create a client for querying the index
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=index_creds)
## Create an index
index_client = SearchIndexClient(service_endpoint, index_creds)

# Azure Openai Settings
openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.azure_endpoint = os.getenv("OPENAI_API_ENDPOINT")
openai.api_version = os.getenv("OPENAI_API_VERSION")

azure_openai_client = AzureOpenAI(
    api_key = os.getenv("OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("OPENAI_API_ENDPOINT")
)

In [3]:
def get_embedding(text, model="textembedding"): # model=[Deployment Name], DONOT change this
   text = text.replace("\n", " ")
   return azure_openai_client.embeddings.create(input = [text], model=model).data[0].embedding

sections = []
with open('outputupdated.csv', 'rt', newline='', encoding='utf-8', errors='ignore') as csvfile:
    csvreader = csv.reader(csvfile)
    item_num = 0
    for item in csvreader:
        section = {
            "id": f"{item_num}",
            "Image_name": item[0],
            "Image_path": item[1],
            "Response1": item[2],
            "Response2": item[3],
            "Caption": item[4],
            "Embedding": get_embedding(''.join(item[0:5]))
        }
        item_num += 1
        sections.append(section)
print(f"Finished Indexing: {len(sections)} items in total")


Finished Indexing: 71 items in total


In [4]:
index = SearchIndex(
    name=index_name,
    fields=[
        SimpleField(name="id", type="Edm.String", key=True),
        SearchableField(name="Image_name", type="Edm.String", analyzer_name="standard.lucene", 
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchableField(name="Image_path", type="Edm.String", analyzer_name="standard.lucene",
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchableField(name="Response1", type="Edm.String", analyzer_name="standard.lucene",
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchableField(name="Response2", type="Edm.String", analyzer_name="standard.lucene",
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchableField(name="Caption", type="Edm.String", analyzer_name="standard.lucene",
                        filterable=True, sortable=True, facetable=True, searchable=True),
        SearchField(name="Embedding", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),  
            hidden=False, searchable=True, filterable=False, sortable=False, facetable=False,
            vector_search_dimensions=1536, vector_search_profile_name="my-vector-config"),
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(
            name="my-vector-config",
            algorithm_configuration_name="my-hnsw")
        ],
        algorithms=[
            HnswAlgorithmConfiguration(name="my-hnsw")
        ]
    )
)

index_client.create_or_update_index(index)

In [5]:
results = search_client.upload_documents(documents=sections)
print("Uploading")
succeeded = sum([1 for r in results if r.succeeded])
print(f"Indexed {len(results)} sections, {succeeded} succeeded")
batch = []

Uploading
Indexed 71 sections, 71 succeeded


In [6]:
query = "Sir ho Kai?" #your query keywords

'''Vector Search for Images'''
# query_vector = get_embedding(query)
# r = search_client.search(
#     search_text=None,
#     top=3,
#     vector_queries=[VectorizedQuery(
#         vector=query_vector,
#         fields="Embedding"
#     )]
# )

'''Full text search'''
r = search_client.search(
    search_text=query
)

search_results = []
for result in r:
    print("#########################################")
    print("Source: " + result["Image_name"])
    print("Response1" + result["Response1"])
    print("Response2" + result["Response2"])
    print("Caption" + result["Caption"])
    search_results.append("Source: " + result["Image_name"] + result["Response1"] + result["Response2"] + "; Caption: " + result["Caption"])

#########################################
Source: Sir Ho Kai.jpg
Response1Story: The image depicts a historical photograph of Dr. Ho Kai, who was later known as Sir Kai Ho Kai. He was a barrister and medical doctor and is noted as the founder of the Alice Memorial Hospital and the College of Medicine. He was also an active promoter of the University.

Objects: 
- Photograph (black and white)
- Suit and tie or bow tie
- Medal or decoration
- Caption with English and Chinese text

Events: 
- None specifically shown, but the text indicates significant contributions to medical institutions and education.

Time:
- Late 19th to early 20th century (approximation based on clothing style and historical context)
Response2Objects: 
- Photograph (black and white)
- Suit and tie or bow tie
- Medal or decoration
- Caption with English and Chinese text

Events: 
- None specifically shown, but the text indicates significant contributions to medical institutions and education.

Time:
- Late 19th to ear

In [13]:
'''Not this part on Generating Answer'''

systemMessage = """AI Assistant that helps user to answer questions from sources provided. Be brief in your answers.
                    Answer ONLY with the facts listed in the list of sources below. 
                    If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. 
                    Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. 
                    Use square brackets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
                """

messages = [
    {'role' : 'system', 'content' : systemMessage},
    {'role' : 'user', 'content' : query + "   Source:" + " ".join(search_results)}
]

chat_completion = openai.chat.completions.create(
    model="summer", # Do not edit this. model="deployment_name"
    messages=messages, 
    temperature=0.7, 
    max_tokens=1024, 
    n=1)

chat_content = chat_completion.choices[0].message.content
print(chat_content)

The first library of the University of Hong Kong (HKU) is the East library in the Main Building. It was already in operation by 1917 [East Library, Main Building.jpg].
